<a href="https://colab.research.google.com/github/vicvyvial/ZPI/blob/main/MEDICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏥 Medica+ – System rejestracji pacjentów

Demo aplikacji w środowisku Google Colab z użyciem **Flask + SQLite**.

In [ ]:
!pip install pyngrok

In [ ]:
!pip install flask pyngrok > /dev/null

from flask import Flask, request, render_template_string, redirect, session, url_for
from pyngrok import ngrok
import sqlite3
import threading

# --- Inicjalizacja Flask ---
app = Flask(__name__)
app.secret_key = 'tajny_klucz'

# --- HTML jako stringi ---
add_patient_html = '''
<h2>Dodaj pacjenta</h2>
<form method="POST">
    Imię: <input name="imie"><br>
    Nazwisko: <input name="nazwisko"><br>
    Data urodzenia: <input type="date" name="data_urodzenia"><br>
    PESEL: <input name="pesel"><br>
    Adres: <input name="adres"><br>
    Telefon: <input name="telefon"><br>
    Email (login): <input name="email"><br>
    Historia medyczna: <textarea name="historia_medyczna"></textarea><br>
    <button type="submit">Dodaj</button>
</form>
'''

login_html = '''
<h2>Logowanie</h2>
<form method="POST">
    Login: <input name="login"><br>
    Hasło: <input type="password" name="haslo"><br>
    <button type="submit">Zaloguj</button>
</form>
'''

card_html = '''
<h2>Karta pacjenta</h2>
<p><strong>Imię:</strong> {{ pacjent.imie }}</p>
<p><strong>Nazwisko:</strong> {{ pacjent.nazwisko }}</p>
<p><strong>Data urodzenia:</strong> {{ pacjent.data_urodzenia }}</p>
<p><strong>PESEL:</strong> {{ pacjent.pesel }}</p>
<p><strong>Adres:</strong> {{ pacjent.adres }}</p>
<p><strong>Telefon:</strong> {{ pacjent.telefon }}</p>
<p><strong>Email:</strong> {{ pacjent.email }}</p>
<p><strong>Historia medyczna:</strong> {{ pacjent.historia_medyczna }}</p>
'''

# --- Funkcja tworząca bazę danych ---
def init_db():
    conn = sqlite3.connect('medica.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Pacjent (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        imie TEXT,
        nazwisko TEXT,
        data_urodzenia TEXT,
        pesel TEXT UNIQUE,
        adres TEXT,
        telefon TEXT,
        email TEXT UNIQUE,
        historia_medyczna TEXT
    )''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Uzytkownik (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        id_pacjenta INTEGER,
        login TEXT UNIQUE,
        haslo TEXT,
        rola TEXT,
        FOREIGN KEY (id_pacjenta) REFERENCES Pacjent(id)
    )''')
    conn.commit()
    conn.close()

# --- Routing aplikacji ---
@app.route('/')
def home():
    return redirect(url_for('login'))

@app.route('/add_patient', methods=['GET', 'POST'])
def add_patient():
    if request.method == 'POST':
        data = request.form
        conn = sqlite3.connect('medica.db')
        cursor = conn.cursor()
        cursor.execute('''INSERT INTO Pacjent (imie, nazwisko, data_urodzenia, pesel, adres, telefon, email, historia_medyczna)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                       (data['imie'], data['nazwisko'], data['data_urodzenia'], data['pesel'],
                        data['adres'], data['telefon'], data['email'], data['historia_medyczna']))
        pacjent_id = cursor.lastrowid
        cursor.execute('''INSERT INTO Uzytkownik (id_pacjenta, login, haslo, rola)
                          VALUES (?, ?, ?, ?)''',
                       (pacjent_id, data['email'], '1234', 'pacjent'))
        conn.commit()
        conn.close()
        return redirect(url_for('login'))
    return render_template_string(add_patient_html)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        login = request.form['login']
        haslo = request.form['haslo']
        conn = sqlite3.connect('medica.db')
        cursor = conn.cursor()
        cursor.execute('SELECT id_pacjenta FROM Uzytkownik WHERE login=? AND haslo=? AND rola="pacjent"', (login, haslo))
        user = cursor.fetchone()
        conn.close()
        if user:
            session['id_pacjenta'] = user[0]
            return redirect(url_for('card'))
    return render_template_string(login_html)

@app.route('/card')
def card():
    if 'id_pacjenta' not in session:
        return redirect(url_for('login'))
    conn = sqlite3.connect('medica.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Pacjent WHERE id=?', (session['id_pacjenta'],))
    row = cursor.fetchone()
    conn.close()
    if row:
        keys = ['id', 'imie', 'nazwisko', 'data_urodzenia', 'pesel', 'adres', 'telefon', 'email', 'historia_medyczna']
        pacjent = dict(zip(keys, row))
        return render_template_string(card_html, pacjent=pacjent)
    return "Nie znaleziono pacjenta."

# --- Inicjalizacja bazy ---
init_db()

# --- Konfiguracja ngrok ---
from pyngrok import ngrok
ngrok.set_auth_token("2vmZ3ZFDfUIJRPaQqSVoCU6bnRF_5q2AgJuLbdLHwT39pKdkD")
public_url = ngrok.connect(5000)
print("Twoja aplikacja Flask jest dostępna pod:", public_url)

# --- Uruchomienie aplikacji w osobnym wątku ---
def start_flask():
    app.run()

threading.Thread(target=start_flask).start()


Twoja aplikacja Flask jest dostępna pod: NgrokTunnel: "https://814f-35-245-187-175.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
